In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from peft import AutoPeftModelForCausalLM, AutoPeftModelForSeq2SeqLM
import torch
import torch.nn.functional as F
from langchain.prompts import PromptTemplate
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import nltk
from back.recommender.LexRank import degree_centrality_scores
import pandas as pd
import numpy as np
import re

In [2]:
U = pd.read_csv('cache/vacancies.csv',index_col=0).fillna('')
I = pd.read_csv('cache/courses.csv',index_col=0)

In [3]:
U.iloc[0,1]

'Приглашаем на удаленный формат работы Программист 1С административная поддержка. Обязанности:  Оказывать поддержку и консультировать пользователей по работе с несколькими конфигурациями на платформе 1С 8.3; Анализ запросов и проблем клиентов, поиск решений и координация процесса их решения с разработчиками и другими отделами компании; Обновление типовых конфигураций 1С ЗУП, Бухгалтерия;   Настройка обменов между различными конфигурациями 1С.  Требования:  Опыт программирования в 1С от двух лет; Знание одной или нескольких типовых конфигураций 1С (желательно УТ); Знание языка запросов, СКД, управляемых форм  Мы предлагаем:  Работа в современной стабильной Компании; Интересные задачи, возможность заниматься не только развитием существующего функционала наших систем автоматизации, но и принять участие в создании нового; Официальное трудоустройство (оплата больничных листов, отпусков); Выплата заработной платы 2 раза в месяц; Удобный график работы: гибкое начало рабочего дня, удаленный ре

# Sentence Transformer

In [74]:
embedder = SentenceTransformer("cointegrated/LaBSE-en-ru")

In [75]:
def summurize(document):
    sentences = nltk.sent_tokenize(str(document))
    embeddings = embedder.encode(sentences, convert_to_tensor=True)
    cos_scores = util.cos_sim(embeddings, embeddings).cpu().numpy()
    centrality_scores = degree_centrality_scores(cos_scores, threshold=None)
    most_central_sentence_indices = np.argsort(-centrality_scores)
    return ' '.join([sentences[idx].strip() for idx in most_central_sentence_indices[0:5]])

In [71]:
#cointegrated/LaBSE-en-ru
It = I['title'] + ' ' + I['description'].apply(summurize)
Ut = U['title'] + ' ' + U['description'].apply(summurize) + ' ' + U['skills'] + ' ' + U['roles']
Iv = embedder.encode(It.values, convert_to_tensor=True)
Uv = embedder.encode(Ut.values, convert_to_tensor=True)
U['rec'] = I.iloc[[i[0]['corpus_id'] for i in util.semantic_search(Uv, Iv, top_k=1)]].title.values
U[['title','rec']]

,title,rec
96176670,1С Программист,1C-разработчик: быстрый старт в профессии
94493829,Стажер-программист 1С,1C-разработчик: быстрый старт в профессии
95546610,Стажер-программист 1С,1C-разработчик: быстрый старт в профессии
96425465,Программист 1C,1C-разработчик: быстрый старт в профессии
96099198,Ведущий сетевой инженер оператора связи (ISP),Сетевой инженер
96050769,Младший сетевой инженер,Сетевой инженер
96454961,Сетевой инженер,BI-аналитик
95587447,Сетевой инженер,Сетевой инженер
96428016,Сетевой инженер,Сетевой инженер
96127386,Python/Go developer (middle/middle+),iOS-разработчик


# Shallow Learning

In [72]:
# Работает плохо!!!

stemmer = SnowballStemmer("russian", True)
def preprocess(text):
    text = re.sub('[.,:;\-+?!()\[\]]', '', str(text).lower().replace('/',' ').replace('#','sharp'))
    return ' '.join([ stemmer.stem(word) for word in word_tokenize(text)])

It = (I['title'] + ' ' + I['description'].apply(summurize)).apply(preprocess)
Ut = (U['title'] + ' ' + U['description'].apply(summurize) + ' ' + U['skills'] + ' ' + U['roles']).apply(preprocess)
vectoriser = TfidfVectorizer()
Iv = vectoriser.fit_transform(It)
Uv = vectoriser.transform(Ut)
U['rec'] = I.iloc[cosine_similarity(Iv,Uv).argmax(0)].title.values
U[['title','rec']]

,title,rec
96176670,1С Программист,Бизнес-аналитик: быстрый старт в профессии
94493829,Стажер-программист 1С,Разработчик на C++: быстрый старт в профессии
95546610,Стажер-программист 1С,ИТ-специалист в сфере Data Science
96425465,Программист 1C,Специалист по автоматизированному тестированию
96099198,Ведущий сетевой инженер оператора связи (ISP),Сетевой инженер
96050769,Младший сетевой инженер,Сетевой инженер
96454961,Сетевой инженер,Сетевой инженер
95587447,Сетевой инженер,Сетевой инженер
96428016,Сетевой инженер,Сетевой инженер
96127386,Python/Go developer (middle/middle+),Специалист по автоматизированному тестированию
